# Decorators

### Decorator chains

In [ ]:
import functools
import sys

def deprecated(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Warning: {func.__name__} is deprecated")
        return func(*args, **kwargs)
    return wrapper

def trace(dest=sys.stderr):
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(
                f'{func.__name__} called with args {args}, {kwargs}!',
                file=dest
            )
            return func(*args, **kwargs)
        return wrapper
    return wraps

@trace(sys.stdout)
def f(x, test):
    if test > 1:
        return f(x, test / 2)

f('Hi!', test=42)

In [ ]:
@deprecated
@trace()
def f(x):
    return x

f(1)

### The notify decorator

In [ ]:
# notify: decorator
#  - without parameters: sends a message to the chat
#  - with parameters: sends a message with the provided parameters
import functools
# from telegram_api import send


def notify(*dec_args):
    if len(dec_args) == 1 and callable(dec_args[0]):
        func = dec_args[0]
        message = "defalut message"
    else:
        func = None
        message = dec_args[0]
    
    def wrapper(func):
        @functools.wraps(func)
        def wraps(*args, **kwargs):
            r = func(*args, **kwargs)
            print(message, file=sys.stderr)
            # your code goes here
            return r
        return wraps
    
    if func is not None:
        return wrapper(func)
    else:
        return wrapper


@notify
def foo():
    '''Docstring'''
    print('foo is done')


@notify('hello chat')
def bar():
    print('bar is done')


foo()
foo()
bar()

### Type hints

[Pep 612](https://peps.python.org/pep-0612/): `ParamSpec`, `Concatenate`, `ReturnType`

In [ ]:
from typing import Callable, TypeVar, TextIO, ParamSpec
import sys

R = TypeVar("R")
P = ParamSpec("P")

def trace(dest: TextIO = sys.stderr):
    def wraps(func: Callable[P, R]) -> Callable[P, R]:
        @functools.wraps(func)
        def wrapper(*args: P.args, **kwargs: P.kwargs):
            print(
                f'{func.__name__} called with args {args}, {kwargs}!',
                file=dest
            )
            return func(*args, **kwargs)
        return wrapper
    return wraps


# Context managers

Examples of context managers: `warnings`

In [ ]:
import numpy as np
import warnings

with warnings.catch_warnings(record=True) as w:
    # Cause all warnings to always be triggered.
    warnings.simplefilter("always")
    np.int32(1) / np.int32(0)
    np.log(0)
    
    for warn in w:
        print(warn)

Syntax of the `with` statement

In [ ]:
# nested contexts
with open('file1.txt') as file1, open('file2.txt') as file2:
    do_something(f, s)
# since python 3.11
with (open('file1.txt'), open('file2.txt')) as (file1, file2):
    do_something(f, s)

In [ ]:
# same as above
with first() as f:
    with second as s():
        do_something(f, s)

In [ ]:
with third():  # <as NAME> part as optional
    do_something()

`contextlib.contextmanager` — a convenient way to create context managers

In [ ]:
from contextlib import contextmanager

@contextmanager
def mycm():
    print('before')
    yield 42  # yep, it is a generator
    print('after')
    
with mycm() as r:
    print(f'got {r}')
    
with mycm() as r:
    raise RuntimeError('Oops')
# 'after' is not printed!

But you have to be careful when working with `contextlib.contextmanager`

In [ ]:
from contextlib import contextmanager

@contextmanager
def mycm():
    print('before')
    try:
        yield ValueError("kek")
    finally:
        print('after')

with mycm() as r:
    print(type(r), r)
    raise RuntimeError('Oops')

The `contextlib` module has other useful things as well:\n
- `contextlib.ContextDecorator` — a base class for context managers; they can then be used as decorators for functions\n
- `contextlib.ExitStack` — allows you to work with an unknown-in-advance number of “resources” and manage context managers dynamically\n
- See the documentation

# Iterators

### The second form of the `iter` function

In [ ]:
import io

stream = io.StringIO('abcdefghi')

def read3() -> str:
    return stream.read(3)

In [ ]:
iter(read3, '')  # every __next__ translates to __call__

In [ ]:
for chunk in iter(read3, ''):  # iter(callable, sentinel)
    print(chunk, end=' ')

In [ ]:
from collections.abc import Callable

def make_timer(ticks: int) -> Callable[[], int]:
    
    def timer() -> int:
        nonlocal ticks
        ticks -= 1
        return ticks

    return timer

In [ ]:
timer = make_timer(2)

In [ ]:
timer()

In [ ]:
timer()

In [ ]:
for i in iter(make_timer(10), -1):
    print(i, end=' ')

### The only reliable way to check if an object is iterable

In [ ]:
try:
    iter(object_to_test)
except TypeError:
    # not an iterable
    ...
else:
    # iterable
    ...

# Generators

### Generator chaining

In [ ]:
def sum_of_squares_of_even(iterable: Iterable[int]) -> int:
    sum_ = 0
    for i in iterable:
        if i % 2 != 0:
            continue
        sum_ += i ** 2
    return sum_

In [ ]:
sum_of_squares_of_even(range(10))

In [ ]:
def even(iterable: Iterable[int]) -> list[int]:
    result = []
    for i in iterable:
        if i % 2 != 0:
            continue
        result.append(i)
    return result

In [ ]:
def squares(iterable: Iterable[int]) -> list[int]:
    result = []
    for i in iterable:
        result.append(i ** 2)
    return result

In [ ]:
sum(squares(even(range(10))))

In [ ]:
def even(iterable: Iterable[int]) -> Iterator[int]:
    for elem in iterable:
        if elem % 2 == 0:
            yield elem

In [ ]:
def squares(iterable: Iterable[int]) -> Iterator[int]:
    for elem in iterable:
        yield elem ** 2

In [ ]:
sum(squares(even(range(10))))

A chain of generators lets you decompose an algorithm easily without significant memory overhead.

## Generators: advanced usage

#### Inspired by: http://dabeaz.com/finalgenerator/

In [ ]:
from collections.abc import Iterator, Generator

In [ ]:
def create_generator() -> Iterator[int]:
    yield 5

In [ ]:
def create_generator() -> Generator[int, None, None]:
    yield 5

In [ ]:
def create_duplicator() -> Generator[int, int, None]:
    print('Give me a value, please')
    value = yield
    print(f'Got value: {value}')
    yield value * 2
    print('Finished')

In [ ]:
dublicator = create_duplicator()
next(dublicator)

In [ ]:
dublicator.send(21)

In [ ]:
dublicator.send(100500)

### [yield as an expression](https://docs.python.org/3/reference/simple_stmts.html#yield)

![yield-expr](https://i0.wp.com/storage.googleapis.com/ssivart/super9-blog/priming-generator.png?w=1200&ssl=1)

In [ ]:
def jumping_counter(upto: int) -> Generator[int, int, None]:
    count = 1
    while count <= upto:
        jump = yield count
        count += jump or 1

In [ ]:
generator = jumping_counter(3)

In [ ]:
next(generator)  # equals to .send(None)

In [ ]:
generator.send(2)

In [ ]:
next(generator)

### [throw](https://docs.python.org/3/reference/expressions.html#generator.throw)

In [ ]:
generator = jumping_counter(5)

In [ ]:
next(generator)

In [ ]:
generator.throw(Exception('Good luck!'))

### [close](https://docs.python.org/3/reference/expressions.html#generator.close)

In [ ]:
generator = jumping_counter(5)

In [ ]:
next(generator)

In [ ]:
generator.close()

In [ ]:
next(generator)

### Handling `close`

In [ ]:
def create_generator() -> Iterator[int]:
    while True:
        try:
            yield 42
        except GeneratorExit:  # can't be ignored
            print('Exiting...')
            return

In [ ]:
generator = create_generator()

In [ ]:
next(generator)

In [ ]:
generator.close()

### [@contextmanager](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager)

In [ ]:
from contextlib import contextmanager
import tempfile
import shutil

In [ ]:
@contextmanager
def tempdir():
    dirname = tempfile.mkdtemp()
    try:
        yield dirname
    finally:
        shutil.rmtree(dirname)

In [ ]:
with tempdir() as path:
    print(path)

In [ ]:
# More precise implementation:
# https://github.com/python/cpython/blob/b3f0ceae919c1627094ff628c87184684a5cedd6/Lib/contextlib.py#L142

class _GeneratorContextManager:
    def __init__(self, func, args, kwargs):
        self.gen = func(*args, **kwargs)
    
    def __enter__(self):
        return next(self.gen)

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type is None:
            try:
                next(self.gen)
            except StopIteration:
                return False
            raise RuntimeError("generator didn't stop")
        else:
            try:
                self.gen.throw(exc_type, exc_value, exc_traceback)
            except BaseException:
                return False
            raise RuntimeError("generator didn't stop after throw()")

def contextmanager(func):
    def helper(*args, **kwargs):
        return _GeneratorContextManager(func, args, kwargs)
    return helper

### [yield from](https://peps.python.org/pep-0380/)

In [ ]:
from collections.abc import Iterable
from typing import TypeVar

T = TypeVar('T')

def repeat(times: int, iterable: Iterable[T]) -> T:
    for _ in range(times):
        yield from iterable  # https://www.python.org/dev/peps/pep-0380/

In [ ]:
for elem in repeat(5, [1, 2, 3]):
    print(elem, end=' ')

In [ ]:
def repeat(times: int, iterable: T) -> T:
    for _ in range(times):
        yield iterable

In [ ]:
for elem in repeat(5, [1, 2, 3]):
    print(elem, end=' ')

In [ ]:
from collections.abc import Iterator
from dataclasses import dataclass

@dataclass
class BinaryTreeNode:
    value: int
    left: 'BinaryTreeNode | None' = None
    right: 'BinaryTreeNode | None' = None

    def __iter__(self) -> Iterator[int]:
        yield from self.left or ()
        yield self.value
        yield from self.right or ()

In [ ]:
tree = BinaryTreeNode(
    left=BinaryTreeNode(
        left=BinaryTreeNode(value=1),
        value=2,
    ),
    value=3,
    right=BinaryTreeNode(
        value=4,
        right=BinaryTreeNode(value=5),
    ),
)

In [ ]:
for value in tree:
    print(value, end=' ')

### [return in generators](https://peps.python.org/pep-0255/#specification-return)

In [ ]:
def create_generator() -> Generator[int, None, int]:
    yield 42
    return 21

In [ ]:
generator = create_generator()
next(generator)
next(generator)

In [ ]:
def generator_wrapper():
    result = yield from create_generator()
    print(result)

In [ ]:
list(generator_wrapper())